# ex-04 Compare Spatial Indexing Methods - RTree

When it comes to query spatial data at scale, there’s no concept more useful and important than a spatial index. Spatial indices are a family of algorithms that arrange geometric data for efficient search. For example, check if a point within a polygon(s).

Within the ecosystem of geopandas, there are two spatial indexing methods available:
- [R-Tree](https://en.wikipedia.org/wiki/R-tree)
> R-trees are tree data structures used for spatial access methods, i.e., for indexing multi-dimensional information such as geographical coordinates, rectangles or polygons. 

- [STRTree](https://pygeos.readthedocs.io/en/latest/strtree.html)
> A query-only R-tree created using the Sort-Tile-Recursive (STR) algorithm. For two-dimensional spatial data. The STR packed R-tree is simple to implement and maximizes space utilization; that is, as many leaves as possible are filled to capacity. 

The trees look like as the following image.
<img  src='img/rtree.png'>
Image is downloaded from https://github.com/tidwall/rtree/blob/master/cities.png

## 1 Import libs

In [1]:
import itertools
import numpy as np
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point

import matplotlib.pyplot as plt
%matplotlib inline

It is worth noting:

Once the operationof df_new.sindex finished, the spatial index was created. Now current index is ***STRTree***, which is from PyGEOSSTRTreeIndex. Therefore, the package of pygeos is installed on your computer, the default index is ***STRTree***. 

To use R-Tree, we should disable ***pygeos*** first before reading data. Thus, geopands will apply the rtree as the default spatial index.

In [2]:
gpd.options.use_pygeos = False

## 2 Prepare data

- Read global map

In [3]:
df_new = gpd.read_parquet('data/gadm404_Level1.parquet')

- Make some fake points (1000 points)

In [4]:
s = gpd.GeoSeries(gpd.points_from_xy([-149.955038, -155.623983, 28.174123, 
                                      -53.701103, 10.800129,  116.372710,
                                      174.783448, 175.27606, 147.352620,
                                      -176.474764
                                     ], 
                                     [21.777515, 19.619468,  -32.321729, 
                                      -10.711625,  62.435156, 39.966970,
                                      -36.889234, -37.768188, -42.016550,
                                      -44.021665,
                                     ]))

a = [s] * 100
ss = list(itertools.chain(*a))
ss = gpd.GeoSeries(ss)
len(ss)

1000

## 3 Points in Polygons (PIPs)

- Check Spatial Indexing Method

In [5]:
ssindex = df_new.sindex
ssindex

rtree.index.Index(bounds=[-179.99999999999994, -59.48428000099989, 180.00000000000014, 83.65833300000008], size=70)

#### Check the speed of PIPs

In [6]:
%%time
idx = ssindex.query_bulk(ss, predicate="within").T

CPU times: total: 266 ms
Wall time: 252 ms


The query results only list the points over land.

In [7]:
land_points = ss.iloc[idx[:, 0]]
lons = [ew_point.coords.xy[0][0] for ipt, ew_point in enumerate(land_points)]
lats = [ew_point.coords.xy[1][0] for ipt, ew_point in enumerate(land_points)]

df_land_points = df_new[['State', 'Country']].iloc[idx[:, 1]]
df_land_points['latitude']  = lats
df_land_points['longitude'] = lons

df_land_points.index = idx[:, 0]
df_land_points

,State,Country,latitude,longitude
1,Hawaii,United States,19.619468,-155.623983
2,Eastern Cape,South Africa,-32.321729,28.174123
3,Mato Grosso,Brazil,-10.711625,-53.701103
4,Hedmark,Norway,62.435156,10.800129
5,Beijing,China,39.966970,116.372710
...,...,...,...,...
995,Beijing,China,39.966970,116.372710
996,Auckland,New Zealand,-36.889234,174.783448
997,Waikato,New Zealand,-37.768188,175.276060
998,Tasmania,Australia,-42.016550,147.352620


## Summary

By comparing the speed of points in polygons (PIPs), it can be found that the method of ***RTree*** is much faster than that of ***STRTree***. Therefore, the STRTree is further applied to develop our PIPs web API with FastAPI.

## Refernces and resources

https://fastapi.tiangolo.com/

https://geopandas.org/en/stable/

https://shapely.readthedocs.io/en/stable/manual.html

https://pygeos.readthedocs.io/en/stable/